**Run the following two cells before you begin.**

In [118]:
%autosave 10

Autosaving every 10 seconds


In [119]:
import pandas as pd
import numpy as np

______________________________________________________________________
**First, import your data set and define the sigmoid function.**
<details>
    <summary>Hint:</summary>
    The definition of the sigmoid is $f(x) = \frac{1}{1 + e^{-X}}$.
</details>

In [120]:
# Import the data set
data=pd.read_csv('cleaned_data.csv')

In [121]:
m=data.shape[0]

In [122]:
# Define the sigmoid function
def sigmoid(x):
    z=1/(1+np.exp(-x))
    return z

**Now, create a train/test split (80/20) with `PAY_1` and `LIMIT_BAL` as features and `default payment next month` as values. Use a random state of 24.**

In [148]:
# Create a train/test split
X=data[['PAY_1','LIMIT_BAL']]
y=data['default payment next month']

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=24)
#X_train.shape

______________________________________________________________________
**Next, import LogisticRegression, with the default options, but set the solver to `'liblinear'`.**

In [149]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(solver='liblinear')

______________________________________________________________________
**Now, train on the training data and obtain predicted classes, as well as class probabilities, using the testing data.**

In [150]:
# Fit the logistic regression model on training data
model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [157]:
# Make predictions using `.predict()`
y_pred=model.predict(X_test)


In [152]:
# Find class probabilities using `.predict_proba()`
y_pred_prob = model.predict_proba(X_test)
pos_prob = y_pred_prob[:,1]
pos_prob

array([0.25173076, 0.415703  , 0.20395547, ..., 0.415703  , 0.17278502,
       0.33606565])

______________________________________________________________________
**Then, pull out the coefficients and intercept from the trained model and manually calculate predicted probabilities. You'll need to add a column of 1s to your features, to multiply by the intercept.**

In [161]:
#Add column of ones to features
features=np.hstack([np.ones((X_test.shape[0],1)), X_test])

In [186]:
# Get coefficients and intercepts from trained model
theta_1 = model.coef_[0][0]
theta_2 = model.coef_[0][1]
#print(theta_1, theta_2)
theta_0 = model.intercept_
#print(theta_0)
theta_values=np.concatenate([model.intercept_.reshape(1,1), model.coef_], axis=1)

In [187]:
# Manually calculate predicted probabilities
Z=np.dot(theta_values,np.transpose(features))
y_pred_prob_manual=sigmoid(Z)
y_pred_prob_manual.shape

(1, 5333)

______________________________________________________________________
**Next, using a threshold of `0.5`, manually calculate predicted classes. Compare this to the class predictions output by scikit-learn.**

In [188]:
# Manually calculate predicted classes
y_pred_manual=y_pred_prob_manual>=0.5

In [195]:
# Compare to scikit-learn's predicted classes
print('manual:',y_pred_manual)
print('using scikit-learn:',y_pred)

#print(y_pred.reshape(1,-1).shape)
#print(y_pred_manual.shape)
np.array_equal(y_pred.reshape(1,-1),y_pred_manual)

manual: [[False False False ... False False False]]
using scikit-learn: [0 0 0 ... 0 0 0]


True

______________________________________________________________________
**Finally, calculate ROC AUC using both scikit-learn's predicted probabilities, and your manually predicted probabilities, and compare.**

In [196]:
# Use scikit-learn's predicted probabilities to calculate ROC AUC
from sklearn import metrics
metrics.roc_auc_score(y_test,y_pred_prob[:,1])

0.627207450280691

In [203]:
# Use manually calculated predicted probabilities to calculate ROC AUC
metrics.roc_auc_score(y_test,y_pred_prob_manual.reshape(y_pred_prob_manual.shape[1]),)

0.627207450280691